# Upstox Market Data Analysis Pipeline
This notebook consolidates the logic for:
1. **Discovery:** Finding Instrument Keys for Nifty 50 & BSE Indices.
2. **History:** Fetching 1-Year Monthly Candle Data.
3. **Live:** Fetching Real-Time LTP.

In [10]:
import os
import time
import requests
import gzip
import json
import io
import pandas as pd
import upstox_client
from dotenv import load_dotenv

# Load secrets
load_dotenv()
ACCESS_TOKEN = os.getenv("UPSTOX_ACCESS_TOKEN")

# Configure APIs
configuration = upstox_client.Configuration()
configuration.access_token = ACCESS_TOKEN
api_client = upstox_client.ApiClient(configuration)

# Initialize BOTH APIs
history_api = upstox_client.HistoryV3Api(api_client)
# Note: User requested MarketQuoteV3Api logic, using MarketQuoteApi as standard SDK class
market_api = upstox_client.MarketQuoteV3Api(api_client)

print("APIs Initialized.")

APIs Initialized.


## Step 1: Instrument Discovery

In [4]:
NIFTY_50_SYMBOLS = [
    "RELIANCE", "TCS", "HDFC", "INFY", "ITC", "LT", "HDFCBANK", "ICICIBANK", "KOTAKBANK", "SBIN",
    "BHARTIARTL", "ASIANPAINT", "MARUTI", "TITAN", "BAJFINANCE", "HCLTECH", "ADANIENT", "SUNPHARMA",
    "ONGC", "NTPC", "TATAMOTORS", "POWERGRID", "ULTRACEMCO", "JSWSTEEL", "TATASTEEL", "M&M",
    "WIPRO", "COALINDIA", "BAJAJFINSV", "BPCL", "NESTLEIND", "BRITANNIA", "TECHM", "EICHERMOT",
    "ADANIPORTS", "GRASIM", "HINDALCO", "DRREDDY", "DIVISLAB", "CIPLA", "APOLLOHOSP", "TATACONSUM",
    "INDUSINDBK", "UPL", "HEROMOTOCO", "BAJAJ-AUTO", "HDFCLIFE", "SBILIFE", "AXISBANK", "LTIM"
]

def get_instrument_keys():
    # 1. Download NSE Master
    print("Downloading NSE Master...")
    nse_resp = requests.get("https://assets.upstox.com/market-quote/instruments/exchange/NSE.json.gz")
    with gzip.GzipFile(fileobj=io.BytesIO(nse_resp.content)) as f:
        nse_data = json.load(f)
        
    # 2. Download BSE Master
    print("Downloading BSE Master...")
    bse_resp = requests.get("https://assets.upstox.com/market-quote/instruments/exchange/BSE.json.gz")
    with gzip.GzipFile(fileobj=io.BytesIO(bse_resp.content)) as f:
        bse_data = json.load(f)
        
    keys = []
    
    # Filter Nifty 50 Stocks
    for item in nse_data:
        if item['segment'] == 'NSE_EQ' and item['trading_symbol'] in NIFTY_50_SYMBOLS:
            keys.append(item['instrument_key'])
            
    # Filter ALL BSE Indices
    for item in bse_data:
        if item['segment'] == 'BSE_INDEX':
            keys.append(item['instrument_key'])
            
    return list(set(keys)) # Remove duplicates

target_keys = get_instrument_keys()
print(f"Target Instruments: {len(target_keys)}")

Target Instruments: 117


## Step 2: Fetch Historical Data (Monthly Candles)

In [7]:
candle_data_store = []

print("Fetching candles")
for key in target_keys:
    try:
        response = history_api.get_historical_candle_data1(
            key,
            "months", 
            "1", 
            "2026-02-06", # Today
            "2025-01-01"  # Start Date
        )
        
        candle_data_store.append({
            "instrument_key": key,
            "candles": response.data.candles
        })
            
    except Exception as e:
        print(f"Failed: {key} - {e}")
        

print(f"Done. History for {len(candle_data_store)} instruments.")

Fetching candles
Done. History for 117 instruments.


## Step 3: Fetch Live LTP (Loop)

In [55]:
ltp_with_names = {}

print("Fetching Live LTP...")

for instrument_key in target_keys:
    
    try:
        # For a single instrument
        response = market_api.get_ltp(instrument_key=instrument_key)
        actual_keys = list(response.data.keys())
        ltp_with_names[instrument_key] = response.data[actual_keys[0]].last_price

    except Exception as e:
        # Using generic Exception to catch all
        print(f"Exception when calling MarketQuoteApi->get_ltp: {e}")

print(f"Fetched LTP for {len(ltp_with_names)} instruments.")

Fetching Live LTP...
Fetched LTP for 117 instruments.


In [56]:
ltp_with_names

{'BSE_INDEX|BBGEFS': 4510.53,
 'BSE_INDEX|BSEHC': 42158.43,
 'BSE_INDEX|SNXN30': 40244.05,
 'BSE_INDEX|FOCMID': 24343.74,
 'NSE_EQ|INE467B01029': 2991.5,
 'BSE_INDEX|BSEFMC': 18622.19,
 'BSE_INDEX|INDSTR': 14661.68,
 'BSE_INDEX|METAL': 39222.37,
 'NSE_EQ|INE095A01012': 913.5,
 'BSE_INDEX|SMEIPO': 87861.85,
 'NSE_EQ|INE437A01024': 7122.0,
 'BSE_INDEX|INSLDR': 15119.61,
 'NSE_EQ|INE075A01022': 233.39,
 'NSE_EQ|INE021A01026': 2432.1,
 'BSE_INDEX|PSUBNK': 5031.95,
 'BSE_INDEX|TELCOM': 3014.29,
 'BSE_INDEX|BSEDSI': 1078.16,
 'BSE_INDEX|PRECON': 5234.23,
 'NSE_EQ|INE522F01014': 431.85,
 'NSE_EQ|INE002A01018': 1443.4,
 'NSE_EQ|INE089A01031': 1244.9,
 'NSE_EQ|INE585B01010': 15059.0,
 'BSE_INDEX|BANKEX': 67536.37,
 'BSE_INDEX|LCTMCI': 9480.83,
 'BSE_INDEX|CAPINS': 2390.38,
 'BSE_INDEX|INFRA': 588.01,
 'BSE_INDEX|BSE500': 36695.11,
 'BSE_INDEX|BSESER': 1543.65,
 'BSE_INDEX|SS6535': 35002.66,
 'NSE_EQ|INE038A01020': 935.45,
 'NSE_EQ|INE009A01021': 1520.2,
 'BSE_INDEX|INTECO': 2979.49,
 'BSE_INDEX

## Now we have ltp and candle data from upstox for 117 instruments now we ned to map it and make a clean prompt for LLM

In [83]:
prompt = "Analyze these stocks,given amount and time to invest\n\n amount :50K and time is 3 months\n\n\n mae sure you use few words in 2 lines"

for candle_data in candle_data_store:
    key = candle_data['instrument_key']
    candles = candle_data['candles']
    ltp = ltp_with_names[key] # Current Price
    
    # 1. Get History
    closes = [c[4] for c in candles] # Extract just the Close prices
    
    # 2. Calculate Long Term Return (1 Year)
    # closes[-1] is the oldest candle (Jan 2025)
    start_price = closes[-1]
    ret_1y = ((ltp - start_price) / start_price) * 100
    
    # 3. Simple Trend (Current vs 3-Month Avg)
    avg_3m = sum(closes[:3]) / 3
    trend = "BULLISH" if ltp > avg_3m else "BEARISH"
    
    # 4. Add to Prompt (Clean Format)
    prompt += f"Stock: {key}\n"
    prompt += f"Price: {ltp} | 1Y Return: {ret_1y:.2f}%\n"
    prompt += f"Trend: {trend} (vs 3M Avg: {avg_3m:.2f})\n\n"
    
print(prompt)

Analyze these stocks,given amount and time to invest

 amount :50K and time is 3 months


 mae sure you use few words in 2 linesStock: BSE_INDEX|BBGEFS
Price: 4510.53 | 1Y Return: 11.21%
Trend: BULLISH (vs 3M Avg: 4498.05)

Stock: BSE_INDEX|BSEHC
Price: 42158.43 | 1Y Return: 0.85%
Trend: BEARISH (vs 3M Avg: 42436.03)

Stock: BSE_INDEX|SNXN30
Price: 40244.05 | 1Y Return: 13.56%
Trend: BEARISH (vs 3M Avg: 40250.18)

Stock: BSE_INDEX|FOCMID
Price: 24343.74 | 1Y Return: 9.39%
Trend: BEARISH (vs 3M Avg: 24351.09)

Stock: NSE_EQ|INE467B01029
Price: 2991.5 | 1Y Return: -27.26%
Trend: BEARISH (vs 3M Avg: 3107.20)

Stock: BSE_INDEX|BSEFMC
Price: 18622.19 | 1Y Return: -9.40%
Trend: BEARISH (vs 3M Avg: 19235.16)

Stock: BSE_INDEX|INDSTR
Price: 14661.68 | 1Y Return: 6.00%
Trend: BULLISH (vs 3M Avg: 14589.71)

Stock: BSE_INDEX|METAL
Price: 39222.37 | 1Y Return: 37.29%
Trend: BULLISH (vs 3M Avg: 38293.09)

Stock: NSE_EQ|INE095A01012
Price: 913.5 | 1Y Return: -7.84%
Trend: BULLISH (vs 3M Avg: 891.27)

In [84]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [85]:
llm=ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=os.getenv("GOOGLE_API_KEY"),
    temperature=0.3,
)

In [86]:
llm.invoke(prompt).content

'Given the short 3-month timeframe, focus on bullish stocks with positive 1Y returns.\n\nAllocate funds to PSUBNK, METAL, INE038A01020, INE123W01016, INE081A01020, INE742F01042, ENERGY, and BSEEVI for potential short-term gains.'